# Data Engine und Dataset

## Data Engine

`````{admonition} Data Engine
:class: note
System, das qualitative, diverse und große **Datasets** für KI Modelle **erstellt**
`````

### Data Engine für SAM

`````{admonition} Problem
:class: warning
**Datenbeschaffung** &rarr; Wenig Bilder mit annotierten Masken online verfügbar.
`````

`````{admonition} Lösung
:class: tip
**Datensätze erstellen** und **Modell co-entwickeln** <br>
    &rarr; Datensätze werden während der Entwicklung des Modells annotiert
    &rarr; "Model-in-the-loop"
`````


### Aufbau 

**3 Phasen:**
1. Assists Manual
2. Semi Automatic
3. Fully Automatic

#### 1 Assists Manual
&rarr; **menschliche Annotators labeln manuell Masken**

- Browser-based **interactive segmentation tool**
    - real-time app powered by SAM
    - Menschen **kennzeichnen Masken**, indem sie auf **Punkte von Objekten** im Vorder-/Hintergrund klicken
- benennbare oder beschreibbare Objekte
- keine semantischen Einschränkungen
- kein Speichern der Labels/Beschreibungen, nur der Punkte und Masken

`````{admonition} Interactive Segmentation
:class: note
... nimmt **Bilder** und **Nutzeranmerkungen** als Input, um Masken von Nutzerinteresse zu erzeugen &rarr; **"Human-in-the-Loop-Methode"** <br>
**Supervised Learning** &rarr; Nutzer verfeinert Segmentierungsergebnisse mit zusätzlichen Informationen {cite}`hao2022eiseg`
`````

##### Training von SAM
1. mit öffentlichen Segmentation Datasets
2. neu annotierte Masken 
    - 6 mal trainiert mit neuen Masken
    - Upgrade bei Image Encoder von ViT-B auf ViT-H <br>
&rarr; **Inferenz Zeit** pro Maske: von 34 Sekunden auf **14 Sekunden** <br>
&rarr; **Masken pro Bild** von 20 auf **44 Masken**

**Insgesamt in dieser Phase &rarr; 4,3 Milionen Masken von 120 K Bildern**

#### 2 Semi Automatic

&rarr; **Diversität der Masken für bessere Segmentierung erhöhen**

- automatische Erkennung von verlässlichen Masken mit SAM
    - Bounding-Box Detector basierend auf Masken der Phase 1 mit "Object" Kategorie
- menschliche Annotator ergänzen weitere Masken

##### Training von SAM
- 5 mal trainiert mit neuen Masken
&rarr; **Inferenz Zeit** pro Maske: wieder auf **34 Sekunden**, wegen höherer Komplexität <br>
&rarr; **Masken pro Bild** von 44 auf **72 Masken**

**Insgesamt in dieser Phase &rarr; 5,9 Milionen Masken von 180 K Bildern**

#### 3 Fully Automatic

**&rarr; voll-automatische Annotierung der Masken**

- **genug Masken** der vorherigen Phasen
- **ambuiguity-aware** Modell entwickelt

##### Mehrdeutigkeits-Modell
- 32 x 32 Grid an Punkten
- für jeden Punkt ein Set an Masken für valide Objekte generieren
- wenn Punkt auf Teil oder Teilbereich eines Objekts
    &rarr; Model gibt Teilbereich, den Teil und gesamtes Objekt zurück
- nur *confident* Masken
- Duplikate rausfiltern

<br>

**Insgesamt &rarr; 1,1 Miliarde Masken von 11 Milionen K Bildern**

### Qualität der Masken
- Masken wurden mit 500 **randomly sampled Bildern** (~50k Masken) getestet
- menschliche Annotators sollten automatischen Masken verbessern
- Vergleich von Paar mit automatisch erkannten und professionell annotierten Masken
    &rarr; IoU für jedes Paar &rarr; **94% der Paare haben höher als 90% IoU**

```{figure} ./images/grid_video.gif
:name: grid_video
Showcase of point grid for automatically segmenting everything in an image. {cite}`AI_2023`
```

---

## Dataset SA-1B

- Verfügbar unter https://ai.facebook.com/datasets/segment-anything/
- 1,1 Milliarde Masken von 11 Millionen Bildern
- wurde komplett automatisch mit der Data Engine gesammelt


### Bilder

- **11 Milionen Bilder** von gewerblichem Anbieter
- High Resolution (3300x4950) Bilder &rarr; 
    - Problem von Accessibility und Storage
    - Bilder deswegen **downsampled**
- auch nach downsampling, haben die Bilder in SA-1B **viel höhere Auflösung als in anderen bekannten Datensätzen**


### Masken

- **1,1 Miliarden Masken**
- **99,1% von SAM generiert**
- verglichen mit professionellen Annotations und anderen Datasets  

<br>


```{figure} ./images/dataset_example_1.png
:name: dataset_example_1
:align: center
Beispielbilder des SA-1B Datasets für ein Bild mit bis zu 50 Masken {cite}`kirillov2023segment`.
```

```{figure} ./images/dataset_example_2.png
:name: dataset_example_2
:align: center
Beispielbilder des SA-1B Datasets für ein Bild mit über 500 Masken {cite}`kirillov2023segment`.
```

---

## Responsible AI Analysis

**&rarr; Untersuchung möglicher Fairnessprobleme und Verzerrungen bei der Verwendung von SA-1B und SAM**

### Geografische Repräsentation
```{figure} ./images/responsible_ai_1.png
:name: responsible_ai_1
:align: center
Geschätzte geografische Verteilung der SA-1B-Bilder. {cite}`kirillov2023segment`.
```

### Fairness bei der Segmentierung von Menschen
```{figure} ./images/responsible_ai_2.png
:name: responsible_ai_2
:width: 400px
height: 300px
:align: center
SAMs Performance bei der Segmentierung von Menschen anhand wahrgenommener Geschlechtszugehörigkeit, Altersgruppe und Hautfarbe. {cite}`kirillov2023segment`.
```